<a href="https://colab.research.google.com/github/mrnithesh/Nila-AI-Therapist/blob/main/Nila_AI_Therapist.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install gtts
import pathlib
import textwrap
import google.generativeai as genai
from google.colab import userdata
from IPython.display import display, Markdown, Audio
from gtts import gTTS
import io
import re

def to_markdown(text):
    text = text.replace('•', '  *')
    return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

def preprocess_text(text):
    """Removes special characters like asterisks and hashtags before TTS."""
    pattern = r"[^\w\s]"
    return re.sub(pattern, "", text)

def generate_audio(text):
    """Generate and play TTS audio from text."""
    # Create an in-memory file-like object
    fp = io.BytesIO()
    # Generate and write audio to the in-memory file
    tts = gTTS(text=text, lang="en")
    tts.write_to_fp(fp)
    # Seek back to the beginning of the file
    fp.seek(0)
    # Play the audio using IPython.display.Audio
    display(Audio(data=fp.read(), autoplay=True))
    # Close the temporary file
    fp.close()

def convo(query):
    response = chat.send_message(query)
    updated_response = to_markdown(response.text)
    display(updated_response)
    # Text-to-speech conversion
    text_to_speech = preprocess_text(updated_response.data)
    generate_audio(text_to_speech)

# Retrieve Google API Key
GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')
genai.configure(api_key=GOOGLE_API_KEY)

# Set up the model
generation_config = {
    "temperature": 1,
    "top_p": 0.95,
    "top_k": 0,
    "max_output_tokens": 8192,
}

safety_settings = [
    {"category": "HARM_CATEGORY_HARASSMENT", "threshold": "BLOCK_MEDIUM_AND_ABOVE"},
    {"category": "HARM_CATEGORY_HATE_SPEECH", "threshold": "BLOCK_MEDIUM_AND_ABOVE"},
    {"category": "HARM_CATEGORY_SEXUALLY_EXPLICIT", "threshold": "BLOCK_MEDIUM_AND_ABOVE"},
    {"category": "HARM_CATEGORY_DANGEROUS_CONTENT", "threshold": "BLOCK_MEDIUM_AND_ABOVE"},
]

system_instruction = (
    "Now you are an AI personal relationship and life counselor and a therapist named Nila. "
    "Act accordingly and give the user advice about their life and personal relationships, "
    "follow a human tone.\n"
)

model = genai.GenerativeModel(
    model_name="gemini-1.5-pro-latest",
    generation_config=generation_config,
    system_instruction=system_instruction,
    safety_settings=safety_settings
)

chat = model.start_chat(history=[])

# Initialize conversation with system instruction
convo(system_instruction)

# Main interaction loop
while True:
    query = input("What's on your mind? :")
    if query.lower() in {"exit", "quit", "bye"}:
        print("Goodbye! Have a great day!")
        break
    convo(query)
